# LTT

Land Transaction Tax (LTT) is a property transaction tax administrated by the Welsh Revenue Authority (WRA) in Wales. It replaced the UK's Stamp Duty Land Tax (SDLT) for property transactions in Wales starting from April 1, 2018. Similar to SDLT, LTT is imposed on property or land transactions, including purchases, transfers, and leases, with values over a certain threshold. It applies to both residential and non-residential (commercial) properties. 

Land Transaction Tax parameters can be found in `policyengine_uk/parameters/gov/wra/land_transaction_tax` and logic in `policyengine_uk/variables/gov/wra/land_transaction_tax.py`.

## Legislation

The original regulation of LTT is defined in [The Land Transaction Tax (Tax Bands and Tax Rates) (Wales) Regulations 2018](https://www.legislation.gov.uk/wsi/2018/128/schedule). It was amended in 2020 by [The Land Transaction Tax (Tax Bands and Tax Rates) (Wales) (Amendment) Regulations 2020](https://www.legislation.gov.uk/wsi/2020/1618/regulation/6/made). The Amendment revised the tax bands and tax rates defined in the 2018 Regulations.

## Rates

Similar to SDLT, the rates of LTT are structured with different threshold and tax bands based on the price of the property. The LTT rates vary for each band, with higher rates applied to properties with prices that exceed a particular threshold. The rates and thresholds also differ for purchase of residential and non-residential properties and rent. Buyers will pay at a higher LTT rate for non-primary residential properties. 

But differing from SDLT. there is no first-time buyers' relief in LTT. Therefore, we're not plotting marginal LTT rate here.

The rules and thresholds for LTT may change over time due to government policy adjustments. Here we show current rates and thresholds for LTT in 2023 below:

- Residential

    - Primary

        Up to £180,000: 0%

        £180,000 to £250,000: 3.5%

        £250,000 to £400,000: 5%
        
        £400,000 to £750,000: 7.5%
        
        £750,000 to £1,500,000: 10%
        
        Over £1,500,000: 12%
    - Non-primary: An additional 4% on top of the rates for primary properties

        Up to £180,000: 4%
        
        £180,000 to £250,000: 7.5%
        
        £250,000 to £400,000: 9%
        
        £400,000 to £750,000: 11.5%
        
        £750,000 to £1,500,000: 14%
        
        Over £1,500,000: 16%

- Non-residential

    Up to £225,000: 0%
    
    £225,000 to £250,000: 1%
    
    £250,000 to £1,000,000: 5%
    
    Over £1,000,000: 6%

- Rent

    Up to £225,000: 0%
    
    £225,000 to £2,000,000: 1%
    
    Over £2,000,000: 2%


In [49]:
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
from policyengine_uk import CountryTaxBenefitSystem

system = CountryTaxBenefitSystem()
wra = system.parameters.gov.wra.land_transaction_tax

parameters = [
    wra.residential.primary,
    wra.residential.higher_rate,
    wra.non_residential,
    wra.rent,
]
thresholds = []
rates = []
labels = []

for parameter in parameters:
    for param in parameter.brackets:
        thresholds.append(param.threshold.values_list[0].value)
        rates.append(param.rate.values_list[0].value)
        labels.append(parameter.metadata.get("label"))

# add a row showing tax rate when property price is higher than the largest threshold for each property type
price = max(thresholds) + 200000
for parameter in parameters:
    thresholds.append(price)
    rates.append(parameter.brackets[-1].rate.values_list[0].value)
    labels.append(parameter.metadata.get("label"))
#
df = pd.DataFrame({"Threshold": thresholds, "Rate": rates, "Label": labels})

# plot
fig = (
    px.line(
        df,
        x="Threshold",
        y="Rate",
        color="Label",
        title="Land Transaction Tax (LTT) rates over property price thresholds",
    )
    .update_layout(yaxis_tickformat=",.0%", xaxis_tickprefix="£", legend_title="")
    .update_traces(line_shape="hv")
)
fig = format_fig(fig)
fig